# Challenge 2 - Resolving Pricing Performance Issues

## 1️⃣ Intro
### 🕵️‍♀️ Investigate Listings with Pricing Performance Issues

Hello, fellow SP-API experts!


I am writing to you on behalf of the 🪄ListingsWizard company. <br />
Our team implemented a new pricing analytics tool based on the data kiosk learnings from our previous incident. <br />
Looking into the analytics tool we noticed that there are some listings not converting well. <br />
Can you please investigate and see how we can improve sales of these listings? My team has provided data access below to assist your investigation.


Yours,
ListingsWizard

---

_* The following scenario uses a mock server and mock data. No real data is used._

### Your Quest
1. Analyze data 📈
2. Identify root cause 👀
3. Submit a fix 🔧

On correct submission, you will receive a passcode. Raise your hand if you receive the passcode 🖐️

First person to receive the correct passcode will be the 🪄 **Wizard of Glory** 💫💫💫


## 2️⃣ Setup
### 🧰 Installing Selling Partner API SDK

The **Amazon Selling Partner API SDK** is an official Amazon library that simplifies integration with Amazon’s SP-API. It handles things like authentication, request signing, and response parsing — so you can focus on building your solution instead of dealing with low-level API mechanics.

With this SDK, you can easily access endpoints like Orders, Listings, Reports, Feeds, and **Buy Shipping** using clean, structured Python code.

📚 Learn more or explore the source on GitHub: https://github.com/amzn/selling-partner-api-sdk/tree/main


📦 To install the SDK, run the below block 👇

In [ ]:
pip install amzn-sp-api

---

## 3️⃣ Let's Data Kiosk!

Data Kiosk lets you submit GraphQL queries from a variety of schemas to help manage selling partner businesses. You can use the following SDK to call Data Kiosk API.


### 🔍 What is Data Kiosk?

It’s your all-access pass to query Amazon datasets like sales, traffic, and profitability using GraphQL.

You can:

- Submit custom queries
- Check their status
- Download results
  
🙌 Time to get seller data with the power of GraphQL!



---

### 🐛 Mocking the SDK


Luckily the Amazon Solution Architect team created a mock server that can replicate the API responses without having credentials.

Below you will find `mock_oauth_endpoint` and `mock_endpoint` variables that can be used to skip the credentials.

In [ ]:
from spapi.models.datakiosk_v2023_11_15 import GetDocumentResponse, CreateQueryResponse, GetQueriesResponse, Query
from spapi.models.product_pricing_v2022_05_01 import CompetitiveSummaryRequest, CompetitiveSummaryResponse, CompetitiveSummaryIncludedData
from spapi.models.listings_items_v2021_08_01 import patch_operation, listings_item_patch_request
from spapi import SPAPIConfig, SPAPIClient, ApiException, QueriesApi, ProductPricingV2022Api, ListingsApi
import json

# Configuration for mock SP-API
mock_oauth_endpoint = "http://54.173.241.222/auth/o2/token"
mock_endpoint = "http://54.173.241.222:80"

config = SPAPIConfig(
    client_id="Chris",
    client_secret="Khoury",
    refresh_token="FakeToken",
    region="NA"
)
client = SPAPIClient(config, endpoint=mock_endpoint, oauth_endpoint=mock_oauth_endpoint)

data_kiosk_api = QueriesApi(client.api_client)
pricing_api = ProductPricingV2022Api(client.api_client)
listing_api= ListingsApi(client.api_client)

print("Connected to SP-API")


---

### 🚶‍♂️ Steps to Get Data Kiosk Reports

#### 🖥️ Step 1: Paste and create Query Below

💡 Use the [Data Kiosk Schema Explorer UI](https://sellercentral.amazon.com/datakiosk-schema-explorer?schema=analytics_salesAndTraffic_2024_04_24) to build your perfect query, for now, you can use the one we built for you here:

In [ ]:
graphql_query_str = (
    "query {analytics_salesAndTraffic_2024_04_24 "
    "{salesAndTrafficByAsin(aggregateBy:CHILD,endDate:\"2025-06-06\",startDate:\"2025-03-01\",marketplaceIds:[\"ATVPDKIKX0DER\"]) "
    "{sales {orderedProductSales {amount currencyCode} totalOrderItems unitsOrdered} "
    "traffic {pageViews buyBoxPercentage} sku childAsin}}}"
)

graphql_query = {"query": graphql_query_str}

# Tell Amazon to start processing your query
create_query_response = data_kiosk_api.create_query(body=graphql_query)
print("✅ Query submitted! Response:", create_query_response)

---

#### 📦 Step 4: Check If Your Data Is Ready to be fetched!

Once the query is ready, you’ll receive a `dataDocumentId`. Use it to grab the results with `get_document` endpoint.

In [ ]:
# Check the status of your query
get_query_response = data_kiosk_api.get_query(query_id=create_query_response.query_id)
print("⏱️ Query status:", get_query_response)

# Now let's get the actual data!
get_document = data_kiosk_api.get_document(document_id=get_query_response.data_document_id)
print("📊 Here's your data:", get_document)

---

### 📊 Visualize Your ASINs Data


🧠 Let's get our Data Kiosk response and interpret it into a nice [dashboard](https://d179ewl1j38ou.cloudfront.net/dashboard.html)

- This view will help you discover what's going wrong.
- Copy the data kiosk response and paste into the dashboard interpret box.

Make sure you’ve run the cell above so `get_document_response` is available.


---

## 4️⃣ Troubleshooting: 

### 🫆 Clue

Looks like the `VOLLEYBALL` SKU has a high page view but is not converting on sales compared with `TSHIRT` SKU. There must be something wrong.

The developer found they were not using important APIs and notifications to avoid this scenarios as PRICING_HEALTH that might had alerted them once their price was not eligible to compete for the BuyBox anymore and getCompetitiveSummary that could bring more insights earlier

To solve the issue, let's discover together some pricing information for calling the getCompetitiveSummary operation and analyze possible scenarios.



#### 🖥️ Step 1: Build our getCompetitiveSummary request

💡 In order to check relevant prices and offers for the item, we are using `getCompetitiveSummary` to fetch this data. Let's build the request for it:

In [ ]:
GCSBody = ({
    "requests": [
        {
            "asin": "B0CLZHRQK8",
            "marketplaceId": "A2Q3Y263D00KWC",
            "includedData": [
                "featuredBuyingOptions",
                "referencePrices"
            ],
            "lowestPricedOffers": [
            {
                "itemCondition": "New",
                "offerType": "Consumer"
            }
            ],
            "method": "GET",
            "uri": "/products/pricing/2022-05-01/items/competitiveSummary"
        }
    ]
})

In [ ]:
import json

get_competitive_summary_response = pricing_api.get_competitive_summary(requests=GCSBody)

print(json.dumps(get_competitive_summary_response.to_dict(), indent=4, default=str))

#### 🖥️ Step 2: Analyze getCompetitiveSummary response.

💡 Let's check the BuyBox current price and what is the CompetitivePrice for that item indicating when the seller offer is eligible to compete for the BuyBox.

To do that, copy the response from the getCompetitiveSummary above and past into the [Pricing Analyzer](https://d179ewl1j38ou.cloudfront.net/pricing-analyzer.html)



## 🚀 Updating Price

We noticed that our SKU has a price above the `CompetitivePrice` indicating it is disqualified for the `FeaturedOffer`.

You can also try it out using the [Pricing Analyzer](https://d179ewl1j38ou.cloudfront.net/pricing-analyzer.html). Simply copy the getCompetitiveSummary response and paste inside the *Competitive Price Analysis* text box.

Using the `competitivePrice` from the `getCompetitiveSummary` API we can now update the item price using the `patchListingsItem` API.

In [ ]:
patch_body = (
        {
        "productType": "PRODUCT",
        "patches": [
            {
                "op": "replace",
                "path": "/attributes/purchasable_offer",
                "value": [
                    {
                        "currency": "BRL",
                        "audience": "ALL",
                        "marketplace_id": "A2Q3Y263D00KWC",
                        "our_price": [
                            {
                                "schedule": [
                                    {
                                        "value_with_tax": # provide value
                                    }
                                ]
                            }
                        ]
                    }
                ]
            }
        ]
    }
)

In [ ]:
import json

patch_listings_item_request = listing_api.patch_listings_item(seller_id="A2ZPJ4TLUOSWY8",sku="VOLLEYBALL",marketplace_ids=["A2Q3Y263D00KWC"],body=patch_body)

print("📊 Here's your pricing update: \n", json.dumps(patch_listings_item_request.to_dict(), indent=4, default=str))